In [1]:
import torch
import random
import torchvision
import numpy as np
from pathlib import Path
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import transforms
from utils.util import make_scheduler

from avalanche.training.supervised import Naive
from avalanche.benchmarks.classic.clear import CLEAR, CLEARMetric
from avalanche.benchmarks.utils import AvalancheDataset
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.plugins.lr_scheduling import LRSchedulerPlugin
from avalanche.logging import InteractiveLogger, WandBLogger
from avalanche.evaluation.metrics import ExperienceAccuracy, ExperienceLoss, ExperienceForgetting, ExperienceCPUUsage, ExperienceMaxGPU, ExperienceMaxRAM, ExperienceTime, EpochAccuracy

/home/cal-05/anaconda3/envs/hspark/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed) # if use multi-GPU
cudnn.deterministic = True  # 연산 처리 속도 감소 -> 모델과 코드를 배포해야 하는 연구 후반 단계에 사용
cudnn.benchmark = False

In [3]:
# For CLEAR dataset setup
DATASET_NAME = "clear100_cvpr2022"
NUM_CLASSES = {"clear10": 11, "clear100_cvpr2022": 100}
assert DATASET_NAME in NUM_CLASSES.keys()

# please refer to paper for discussion on streaming v.s. iid protocol
EVALUATION_PROTOCOL = "streaming"  # trainset = testset per timestamp
# EVALUATION_PROTOCOL = "iid"  # 7:3 trainset_size:testset_size

# For saving the datasets/models/results/log files
ROOT = Path("./data")
DATA_ROOT = ROOT / DATASET_NAME
MODEL_ROOT = ROOT / "models"
DATA_ROOT.mkdir(parents=True, exist_ok=True)
MODEL_ROOT.mkdir(parents=True, exist_ok=True)

normalize = torchvision.transforms.Normalize(
    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
)
train_transform = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize(224),
        torchvision.transforms.RandomCrop(224),
        torchvision.transforms.ToTensor(),
        normalize,
    ]
)
test_transform = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize(224),
        torchvision.transforms.CenterCrop(224),
        torchvision.transforms.ToTensor(),
        normalize,
    ]
)

In [10]:
interactive_logger = InteractiveLogger()
wandb_logger = WandBLogger(run_name="streaming-CLEAR")
eval_plugin = EvaluationPlugin(
    EpochAccuracy(),
    ExperienceAccuracy(),
    ExperienceLoss(),
    ExperienceForgetting(),
    ExperienceCPUUsage(),
    ExperienceMaxGPU(gpu_id=0),
    ExperienceMaxRAM(),
    ExperienceTime(),
    loggers=[interactive_logger, wandb_logger])

/home/cal-05/anaconda3/envs/hspark/lib/python3.8/site-packages/avalanche/training/plugins/evaluation.py:81: UserWarning: No benchmark provided to the evaluation plugin. Metrics may be computed on inconsistent portion of streams, use at your own risk.
  warnings.warn(


In [4]:
# Define hyperparameters/scheduler/augmentation
HPARAM = {
    "batch_size": 256,
    'num_epoch' : 100,
    "step_scheduler_decay": 30,
    "scheduler_step": 0.1,
    "start_lr": 0.01,
    "weight_decay": 1e-5,
    "momentum": 0.9,
}

if EVALUATION_PROTOCOL == "streaming":
    seed = None
else:
    seed = 0

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

num_class = 10
incremental = 5
lr_milestones = [49, 63]
lr_factor = 5.0
fixed_class_order = [4, 1, 7, 5, 3, 9, 0, 8, 6, 2]

scenario = CLEAR(
    data_name=DATASET_NAME,
    evaluation_protocol=EVALUATION_PROTOCOL,
    feature_type=None,
    seed=seed,
    train_transform=train_transform,
    eval_transform=test_transform,
    dataset_root=DATA_ROOT,
)

model = torchvision.models.resnet18(pretrained=False)
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=2.0, momentum=0.9, weight_decay=1e-5)

scheduler = make_scheduler(
    optimizer,
    HPARAM["step_scheduler_decay"],
    HPARAM["scheduler_step"],
)

plugin_list = [LRSchedulerPlugin(scheduler)]

data/clear100_cvpr2022/training_folder does not exist. 
CLEAR has not yet been downloaded


--2022-06-10 17:04:59--  https://clear-challenge.s3.us-east-2.amazonaws.com/clear100-workshop-avalanche.zip
Resolving clear-challenge.s3.us-east-2.amazonaws.com (clear-challenge.s3.us-east-2.amazonaws.com)... 3.5.132.11
Connecting to clear-challenge.s3.us-east-2.amazonaws.com (clear-challenge.s3.us-east-2.amazonaws.com)|3.5.132.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4732258858 (4.4G) [application/zip]
Saving to: ‘data/clear100_cvpr2022/clear100-workshop-avalanche.zip.5’

     0K .......... .......... .......... .......... ..........  0%  154K 8h19m
    50K .......... .......... .......... .......... ..........  0% 15.2M 4h12m
   100K .......... .......... .......... .......... ..........  0%  309K 4h11m
   150K .......... .......... .......... .......... ..........  0% 11.0M 3h10m
   200K .......... .......... .......... .......... ..........  0%  158K 4h9m
   250K .......... .......... .......... .......... ..........  0%  115M 3h28m
   300K .....

In [ ]:
memory_size = 2000
train_batch = 64
eval_batch = 32
epoch = 70

cl_strategy = Naive(
    model,
    optimizer,
    torch.nn.CrossEntropyLoss(),
    train_mb_size=HPARAM["batch_size"],
    train_epochs=HPARAM["num_epoch"],
    eval_mb_size=HPARAM["batch_size"],
    evaluator=eval_plugin,
    device=device,
    plugins=plugin_list,
)

pllugins :  [<avalanche.training.plugins.lr_scheduling.LRSchedulerPlugin object at 0x000001FF26150EE0>, <avalanche.training.strategies.icarl._ICaRLPlugin object at 0x000001FF260FC130>, <avalanche.training.losses.ICaRLLossPlugin object at 0x000001FF2457DE20>, <avalanche.training.plugins.evaluation.EvaluationPlugin object at 0x000001FF260FC1F0>]


In [ ]:
print("Starting experiment...")
results = []
print("Current protocol : ", EVALUATION_PROTOCOL)
for index, experience in enumerate(scenario.train_stream):
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)
    res = cl_strategy.train(experience)
    torch.save(
        model.state_dict(),
        str(MODEL_ROOT / f"model{str(index).zfill(2)}.pth")
    )
    print("Training completed")
    print(
        "Computing accuracy on the whole test set with"
        f" {EVALUATION_PROTOCOL} evaluation protocol"
    )
    results.append(cl_strategy.eval(scenario.test_stream))
# generate accuracy matrix
num_timestamp = len(results)
accuracy_matrix = np.zeros((num_timestamp, num_timestamp))
for train_idx in range(num_timestamp):
    for test_idx in range(num_timestamp):
        accuracy_matrix[train_idx][test_idx] = results[train_idx][
            f"Top1_Acc_Stream/eval_phase/test_stream/Task00{test_idx}"]
print('Accuracy_matrix : ')
print(accuracy_matrix)
metric = CLEARMetric().get_metrics(accuracy_matrix)
print(metric)